# ALGOS DE RECO #

In [1]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from scipy import stats

# on importe les différentes librairies surprise de scikit
from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans

# on importe notre fonction de tri
from tri_threshold import filter_reviews

# on importe notre bdd
avis = pd.read_csv("BDD/avis.csv", index_col="Unnamed: 0")
avis_norm = pd.read_csv("BDD/avis_norm.csv")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
avis.head(1)

,author,date_published,title_review,note,title,url,comment
0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Lorsque le jeu est jeu, bon, réflexif, joli po..."


In [ ]:
# Sélection des lignes où la note est égale à 0
jeux_avec_note_zero = avis.loc[avis['note'] == 0]

# Affichage des résultats
print(jeux_avec_note_zero)

               author date_published  \
619            Zypher        2012-12   
620          Manu0801        2018-02   
704             api25        2016-06   
1368           Olives        2019-11   
2186           apiret        2019-12   
...               ...            ...   
174110          DidOo        2011-12   
174366          DidOo        2014-09   
174397           CyrC        2009-06   
174639  the_farbarian        2016-06   
176064          Albes        2016-11   

                                             title_review  note  \
619                                 Fuyez, pauvres fous !     0   
620     Long et incontrôlable, ça existe encore ce typ...     0   
704                                      Long et ennuyeux     0   
1368                                     Vraiment mauvais     0   
2186                                     Vraiment mauvais     0   
...                                                   ...   ...   
174110                           Pour vous la raco

Il faut mettre en place la matrice sparse (calculer la sparsité + plot cette sparsite)

In [ ]:
num_users  = avis["author"].nunique()
num_items  = avis["title"].nunique()

print(f"Il y'a {num_users} auteurs et {num_items} jeux")

sparsity = (len(avis) / (num_users*num_items))*100
print(f"Notre matrice a une sparsité de {sparsity:.2f}% ")

matrice_sparse = csr_matrix(avis['note'])
matrice_sparse



Il y'a 2564 auteurs et 3057 jeux
Notre matrice a une sparsité de 1.60% 


<1x125299 sparse matrix of type '<class 'numpy.int64'>'
	with 124925 stored elements in Compressed Sparse Row format>

On test maintenant les differents algos (SVD + KNN pour commencer)

## SVD ##

In [ ]:
algo = SVD(n_factors=50,n_epochs=20,lr_all=0.005,reg_all=0.05)

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['author'], df['title'], df['note'])]
        self.reader = reader


data_test = MyDataset(avis, reader)
cross_validate(algo, data_test, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8302  1.8393  1.8369  1.8300  1.8236  1.8320  0.0056  
MAE (testset)     1.3995  1.4114  1.4092  1.4042  1.4058  1.4060  0.0041  
Fit time          3.23    3.04    3.40    3.40    3.19    3.25    0.13    
Test time         0.11    0.17    0.12    0.11    0.17    0.14    0.03    


{'test_rmse': array([1.83020469, 1.83930214, 1.83689914, 1.8299808 , 1.82359466]),
 'test_mae': array([1.39953742, 1.41141878, 1.40921506, 1.40424574, 1.40579265]),
 'fit_time': (3.229295492172241,
  3.0393242835998535,
  3.3971614837646484,
  3.3953378200531006,
  3.1927309036254883),
 'test_time': (0.10840439796447754,
  0.17353558540344238,
  0.11569046974182129,
  0.1077117919921875,
  0.17322993278503418)}

In [ ]:
# Define the Reader object specifying the rating scale
reader = Reader(line_format='user item rating', rating_scale=(0,1))

# Custom Dataset class to load our dataset
class MyDataset(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['author'], df['title'], df['note'])]
        self.reader = reader

# Instantiate the dataset with your data
data = MyDataset(avis_norm, reader)

# Initialize the SVD algorithm
algo = SVD()

# Perform cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1944  0.1928  0.1923  0.1925  0.1945  0.1933  0.0010  
MAE (testset)     0.1504  0.1487  0.1485  0.1491  0.1508  0.1495  0.0009  
Fit time          5.62    5.28    5.24    5.30    5.36    5.36    0.13    
Test time         0.11    0.18    0.11    0.18    0.13    0.14    0.03    


{'test_rmse': array([0.194417  , 0.1927808 , 0.1922522 , 0.19250135, 0.19447668]),
 'test_mae': array([0.15043278, 0.14868858, 0.14852469, 0.14910759, 0.15080227]),
 'fit_time': (5.615259647369385,
  5.276290416717529,
  5.242383241653442,
  5.304482698440552,
  5.364598035812378),
 'test_time': (0.11465334892272949,
  0.17752504348754883,
  0.11369609832763672,
  0.17952609062194824,
  0.13364267349243164)}

In [ ]:
# Assuming 'avis' is your DataFrame and you're analyzing the 'note' column for outliers
z_scores = stats.zscore(avis['note'])
abs_z_scores = abs(z_scores)
filtered_entries = (abs_z_scores < 2)  # Adjust the threshold as necessary
cleaned_data = avis[filtered_entries]

In [ ]:
# Define the Reader object specifying the rating scale
reader = Reader(line_format='user item rating', rating_scale=(0,10))

# Custom Dataset class to load our dataset
class MyDataset(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['author'], df['title'], df['note'])]
        self.reader = reader

# Instantiate the dataset with your data
data_clean = MyDataset(cleaned_data, reader)

# Initialize the SVD algorithm
algo = SVD(n_factors=20,n_epochs=30,lr_all=0.005,reg_all=0.2)

# Perform cross-validation
cross_validate(algo, data_clean, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5532  1.5596  1.5363  1.5376  1.5429  1.5459  0.0091  
MAE (testset)     1.2363  1.2391  1.2258  1.2266  1.2280  1.2312  0.0055  
Fit time          3.07    3.03    2.90    2.98    3.28    3.05    0.13    
Test time         0.18    0.11    0.11    0.10    0.11    0.12    0.03    


{'test_rmse': array([1.55319331, 1.55960379, 1.53629005, 1.53755461, 1.54288815]),
 'test_mae': array([1.23631652, 1.23906095, 1.22576629, 1.22660032, 1.22802834]),
 'fit_time': (3.066718816757202,
  3.030341863632202,
  2.897665023803711,
  2.9796957969665527,
  3.2814018726348877),
 'test_time': (0.17548632621765137,
  0.11468982696533203,
  0.10523509979248047,
  0.10272359848022461,
  0.11417174339294434)}

In [ ]:
# Define the parameter grid
param_grid = {
    'n_factors': [50, 100, 150],  # Number of factors
    'n_epochs': [20, 30],         # Number of iterations
    'lr_all': [0.005, 0.01],      # Learning rate
    'reg_all': [0.02, 0.05]       # Regularization term
}

# Setup GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Fit GridSearchCV
gs.fit(data_test)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])


In [ ]:
param_grid = {
    'n_factors': [20, 50, 100, 150, 200],  # More options for the number of factors
    'n_epochs': [5, 10, 20, 30, 50],       # Broader range of iterations
    'lr_all': [0.002, 0.005, 0.01, 0.02],  # Wider range of learning rates
    'reg_all': [0.02, 0.05, 0.1, 0.2],     # Wider range of regularization terms
    'biased': [True, False],               # Whether to use baseline factors or not
}

# Setup GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Fit GridSearchCV
gs.fit(data_clean)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
cleaned_data_processed = filter_reviews(cleaned_data)

In [ ]:
cleaned_data_processed.describe()

In [ ]:
# Define the Reader object specifying the rating scale
reader = Reader(line_format='user item rating', rating_scale=(0,10))

# Custom Dataset class to load our dataset
class MyDataset(dataset.DatasetAutoFolds):
    def __init__(self, df, reader):
        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['author'], df['title'], df['note'])]
        self.reader = reader

# Instantiate the dataset with your data
data_clean_proc = MyDataset(cleaned_data_processed, reader)

# Initialize the SVD algorithm
algo = SVD(n_factors=20,n_epochs=30,lr_all=0.005,reg_all=0.2)

# Perform cross-validation
cross_validate(algo, data_clean_proc, measures=['RMSE', 'MAE'], cv=5, verbose=True)


## k-NN ##

In [ ]:
algo = KNNBasic()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(jeux, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
avis_norm.head(1)